In [1]:
# 관광지 API 추출(1) - contentid, 위도/경도
import requests  # http request
import json
import pandas as pd

# 생략없이 전체 행 보기
pd.set_option('display.max_rows', None)

# TourAPI 인증키 통하여 json 형식으로 호출 (시군구 구분)
# 강원도 시/군구 총 18개

# 시군구 코드
numlist=[]
for i in range(1,19):
    numlist.append(str(i))

# 시군구 관광지 별 추출
listdata=[]
for num in numlist:
    url = "https://apis.data.go.kr/B551011/KorService1/areaBasedList1?serviceKey=2rTA0AsQYrJXYZI8zaF0J2zdy6kUHExuEVXoYPtGJbcUWAkiSpziy2aw%2BVpKwoyE5DFegvPyKwzFTuKcxY16%2Fw%3D%3D&pageNo=1&numOfRows=50&MobileApp=AppTest&MobileOS=ETC&arrange=A&contentTypeId=12&areaCode=32&sigunguCode="+num+"&_type=json"
    r = requests.get(url)
    data = r.json()
    item = data['response']['body']['items']['item']
    
    # 대괄호 제외하고 listdata에 넣기
    if item: 
        for j in range(len(item)):
            listdata.append(item[j])
listdata

# 주소, contentid, 장소명 추출 후 key:value 형태로 변환
newdictdata = []  # 리스트로 초기화

for i in range(len(listdata)):  # 리스트의 길이만큼 반복
    contentid = listdata[i]['contentid']
    title = listdata[i]['title']
    addr = listdata[i]['addr1'] + listdata[i]['addr2']
    mapx = listdata[i]['mapx']
    mapy = listdata[i]['mapy']
    
    # 딕셔너리 형태로 추가
    item_dict = {
        'contentid': contentid,
        '장소명': title,
        '주소': addr,
        '경도': mapx,
        '위도': mapy
    }
    # 리스트에 추가
    newdictdata.append(item_dict) 

# 결과 확인 => list
newdictdata  # 리스트 형태로 출력

# list -> json 문자열 변환
json_str = json.dumps(newdictdata,ensure_ascii=False)

# json 문자열 -> df 변환
dfdata=pd.read_json(json_str) 
# dfdata.head(5)
# dfdata.info() # 위도, 경도 float

# 아래 url의 contentid 작동을 위해 int -> string 
dfdata=dfdata.astype({'contentid':'string'})

# contentid 칼럼 추출
dfdataci=dfdata.iloc[:,0] # series

# 위도/경도 칼럼 추출
dfdataci_map=dfdata.iloc[:,3:]
dfdataci.head(5)
# dfdataci.item()

# dfdataci
# print(type(dfdataci)) # Series
# print(dfdataci.dtype)

0    2511669
1    2776846
2    2900682
3     125790
4     128758
Name: contentid, dtype: string

In [3]:
# 관광지 API 추출(2) - 강원도 시군구/관광지 별 장소명, 주소, 개요 정보

# 위 contentid를 받아서 json 형식으로 호출
# contentid를 conid라는 변수로 받음

listdata2=[]
for index, conid in dfdataci.items():
    url = "http://apis.data.go.kr/B551011/KorService1/detailCommon1?serviceKey=2rTA0AsQYrJXYZI8zaF0J2zdy6kUHExuEVXoYPtGJbcUWAkiSpziy2aw%2BVpKwoyE5DFegvPyKwzFTuKcxY16%2Fw%3D%3D&contentId="+conid+"&defaultYN=Y&addrinfoYN=Y&overviewYN=Y&MobileOS=ETC&MobileApp=AppTest&_type=json"
    r = requests.get(url)
    data2 = r.json()   
    item = data2['response']['body']['items']['item']
    
    # 대괄호 제외하고 listdata2에 넣기
    if item: 
        listdata2.append(item[0])

# TourAPI에서 강원도 관광지 별 장소명, 주소, 개요 정보 가져오기
newdictdata2 = [] 
for i in range(len(listdata2)):  
    title = listdata2[i]['title']
    addr = listdata2[i]['addr1'] + listdata2[i]['addr2']
    overview = listdata2[i]['overview']
    
    # 주소에서 시군구 분류 추출
    sigungu = ''
    if '강릉' in addr:
        sigungu = '강릉시'
    elif '고성' in addr:
        sigungu = '고성군'
    elif '동해' in addr:
        sigungu = '동해시'
    elif '삼척' in addr:
        sigungu = '삼척시'
    elif '속초' in addr:
        sigungu = '속초시'
    elif '양구' in addr:
        sigungu = '양구군'
    elif '양양' in addr:
        sigungu = '양양군'
    elif '영월' in addr:
        sigungu = '영월군'
    elif '원주' in addr:
        sigungu = '원주시'
    elif '인제' in addr:
        sigungu = '인제군'
    elif '정선' in addr:
        sigungu = '정선군' 
    elif '철원' in addr:
        sigungu = '철원군'
    elif '춘천' in addr:
        sigungu = '춘천시'
    elif '태백' in addr:
        sigungu = '태백시'
    elif '평창' in addr:
        sigungu = '평창군'
    elif '홍천' in addr:
        sigungu = '홍천군'
    elif '화천' in addr:
        sigungu = '화천군'
    elif '횡성' in addr:
        sigungu = '횡성군'
    
    # 딕셔너리 형태로 추가
    item_dict = {
        '장소명': title,
        '시군구 분류': sigungu,
        '주소': addr,
        '설명': overview
    }
    
    # 리스트에 추가
    newdictdata2.append(item_dict) 

# 결과 확인 => list
# newdictdata2

# list -> json 문자열 변환
json_str2 = json.dumps(newdictdata2,ensure_ascii=False)

# json 문자열 -> df 변환
dfdata2=pd.read_json(json_str2) 

# 위도/경도 칼럼 추가
dfdata_final=pd.concat([dfdata2,dfdataci_map],axis=1)
dfdata_final

# DB 연동 사전 작업 - list로 변환
all_df2=[]
for i in range(len(dfdata_final)):
    all_df2.append(dfdata_final.loc[i].to_list())

all_df2[:5]

[['갈골한과마을 명인의 집',
  '강릉시',
  '강원특별자치도 강릉시 사천면 중앙서로 62(사천면)',
  '갈골한과마을 명인의 집은 150년 이상의 역사가 있는 강릉 식문화유산이다. 최봉석 명인이 본격적으로 한과를 시작한 것은 1960년대 초이지만 이미 집안에서의 시작은 증조부 때부터이니 벌써 5대째 최봉석 명인이 60년 넘게 가업을 이어오면서 한결같은 한과의 맛을 이어가고 있다. 갈골과줄의 전승 역사는 예절과 효행을 중시하는 가문의 영향으로 유난히 제례 등의 행사가 많았던 것에서 비롯되었다고 한다. \n갈골과줄은 또한 30일의 기다림이라고도 표현하는데, 그 이유는 첫 번째 단계인 찹쌀 발효에만 꼬박 20일이 걸리기 때문이다. 좋은 쌀을 선별해 첨가물을 일절 넣지 않고 공기 중의 건강한 발효균을 통해 자연 발효를 시키는 방식으로 만든다.',
  128.843283178,
  37.824514771],
 ['강남축구공원',
  '강릉시',
  '강원특별자치도 강릉시 남부로 222 (노암동)',
  '강릉 지역 주민들의 건강 증진과 생활 체육 활성화 등의 스포츠 복지를 실현하고 축구의 고장인 강릉에서 전국 규모의 축구 대회를 유치하는 등의 목적으로 세워진 체육 시설이다.  「강릉시 문화 체육 시설 관리 및 운영 조례」에 따라 운영되고 있으며, 최신 시설을 갖추고 있다. 시민 누구나 편리하게 이용할 수 있다.  강남축구공원 내 스쿼시장과 볼링장도 있다.  축구장(인조 잔디)은 유료 이용이고, 사용일 7일 전에 강릉시 체육 시설 관리사무소에 사용신청서를 제출해야 한다. 풋살장, 족구장, 농구장은 무료로 이용할 수 있고 강남축구공원 관리사무실에 사용신청서만 제출하면 된다.',
  128.9074798939,
  37.7436587117],
 ['강릉 3.1운동 기념공원',
  '강릉시',
  '강원특별자치도 강릉시 저동645',
  '경포대 아래 가시연 습지로 가는 길목에 강릉 지역의 독립 만세운동을 기념하는 공원이 조성되어 있다. 경포호 전경을 가장 잘 볼 수

In [4]:
# DB 연동 - 강원도 시군구 별 관광지 정보 insert
import cx_Oracle as cx
con2=cx.connect("hr","hr","localhost:1521/xe") 
cur2=con2.cursor() 
sql2="insert into attraction(place, sigungu, address, overview, longtitude, altitude) values(:1,:2,:3,:4,:5,:6)" 

# insert 내용들
for place, sigungu, address, overview, longtitude, altitude in all_df2:
    cur2.execute(sql2,(place,sigungu,address,overview,longtitude, altitude))

# commit
con2.commit() 